In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'is_train', 'date']

def filter_feature(path):
    if \
    path.count('') \
    :
#     and not path.count('raw__D') \
#     and not path.count('raw__C') \
#     and not path.count('raw__V') \
#     or path.count('Fraud') 
#     or path.count('tionID') 
#     or path.count('C1')
#     or path.count('C13')
#     or path.count('V232')
#     or path.count('addr1')
        return True
    else:
        return False
    

paths_train = glob('../feature/raw_use/*_train.gz')
paths_train = [path for path in paths_train if filter_feature(path) ]
paths_test = glob('../feature/raw_use/*_test.gz')
paths_test = [path for path in paths_test if filter_feature(path) ]

paths_train_feature = sorted(glob('../feature/org_use/*_train.gz'))
paths_test_feature  = sorted(glob('../feature/org_use/*_test.gz'))

paths_train_feature += glob('../feature/valid/60*_train.gz')
paths_test_feature += glob('../feature/valid/60*_test.gz')

df_train = reduce_mem_usage( parallel_load_data(paths_train) )
df_test  = reduce_mem_usage( parallel_load_data(paths_test) )
Y = df_train[COLUMN_TARGET]
df_train.drop(COLUMN_TARGET, axis=1, inplace=True)

valid_paths_train = paths_train_feature[:600]
valid_paths_test  = paths_test_feature[:600]

#========================================================================
# pathの存在チェック。なぜかたびたびFileNotFoundErrorが起きるので,,,
#========================================================================

remove_paths = []
for trn_path, tes_path in zip(valid_paths_train, valid_paths_test):
    if os.path.exists(trn_path) and os.path.exists(tes_path):
        pass
    else:
        remove_paths.append(trn_path)
        remove_paths.append(tes_path)
for path in remove_paths:
    if path.count('train'):
        valid_paths_train.remove(path)
        print(f'remove {path}')
    elif path.count('test'):
        valid_paths_test.remove(path)
        print(f'remove {path}')

df_feat_train = parallel_load_data(valid_paths_train)
df_feat_test  = parallel_load_data(valid_paths_test)

col_drops = eval_check_feature(df_feat_train, df_feat_test)

tmp_train = df_train.join(df_feat_train)
tmp_test = df_test.join(df_feat_test)

Memory usage of dataframe is 58.01 MB
Memory usage after optimization is: 49.56 MB
Decreased by 14.6%
Memory usage of dataframe is 263.84 MB
Memory usage after optimization is: 310.71 MB
Decreased by -17.8%


  0%|          | 2/600 [00:00<00:30, 19.42it/s]

* Check Unique Feature.


100%|██████████| 600/600 [00:09<00:00, 60.53it/s]


  * 2 No Info Features: ['603__card6-addr1_V258__ProductCD-W_std', '603__card6-addr2_V258__ProductCD-W_std']
  * 2feature unique drop and move trush
['603__card6-addr2_V258__ProductCD-W_std', '603__card6-addr1_V258__ProductCD-W_std']


In [6]:
tmp_train.head()

,raw__V294,raw__card3,TransactionID,raw__P_emaildomain_anonymous_com_dummie,raw__V258,raw__card2,raw__D2,raw__C13,raw__R_emaildomain_gmail_com_dummie,raw__D3,...,603__card5-addr1_V315__ProductCD-H_std,603__card6-addr1_C5__ProductCD-S_std,603__card1_V258__ProductCD-C_std,603__card5-addr2_C6__ProductCD-R_std,603__card6-addr1_C13__ProductCD-S_sum,603__card6-addr2_V78__ProductCD-H_std,603__card6-addr2_C11__ProductCD-S_sum,603__card5-addr1_C1__ProductCD-S_std,603__card5-addr1_C6__ProductCD-W_std,603__card5-addr2_C7__ProductCD-H_std
0,1.0,150.0,2987000,0.0,NaN,NaN,NaN,1.0,0.0,13.0,...,0.126483,0.115042,0.000000,0.511458,-8305.0,0.541009,-183114.0,0.000000,21.363230,0.159065
1,0.0,150.0,2987001,0.0,NaN,404.0,NaN,1.0,0.0,NaN,...,8.479987,0.147763,0.054677,36.881973,-17496.0,0.541009,-183114.0,0.274019,17.858667,0.301094
2,0.0,150.0,2987002,0.0,NaN,490.0,NaN,1.0,0.0,NaN,...,1.549975,0.323652,0.000000,8.506442,112554.0,0.260195,-701702.0,2.859007,24.991880,0.114643
3,38.0,150.0,2987003,0.0,NaN,567.0,112.0,25.0,0.0,0.0,...,0.141112,0.105548,0.062672,18.994841,-10355.0,0.260195,-701702.0,0.000000,16.700760,0.178001
4,0.0,150.0,2987004,0.0,1.0,514.0,NaN,1.0,0.0,NaN,...,6.939574,0.086064,0.000000,36.881973,-1835.0,0.541009,-183114.0,0.185293,3.160956,0.301094
